# RealClearPolitics: 2022 Senate polls

#### Import Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_grid as altgrid
import numpy as np
import us
import urllib.request, json
import glob
import os
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import os
import time

In [3]:
alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
pd.set_option("display.max_colwidth", None)

In [5]:
today = pd.to_datetime("today").strftime("%Y-%m-%d")

In [6]:
today

'2022-10-25'

---

## Get data

#### Get the timeseries from 538

In [7]:
states = [
    "North Carolina",
    "Ohio",
    "Wisconsin",
    "Nevada",
    "Georgia",
    "Pennsylvania",
    "Arizona",
    "Colorado",
]
states

['North Carolina',
 'Ohio',
 'Wisconsin',
 'Nevada',
 'Georgia',
 'Pennsylvania',
 'Arizona',
 'Colorado']

#### Use a list comprehension to feed `read_json` from each state. Then concatenate the list of states' data into dataframe, storing the state name for each race

In [ ]:
%%time

df = pd.concat(
    [
        pd.read_json(
            f"https://projects.fivethirtyeight.com/polls/senate/2022/{state.lower().replace(' ', '-')}/polling-average.json"
        ).assign(state_name=state).sort_values(["candidate", "date"], ascending=False)
        for state in states
    ]
).reset_index(drop=True)

CPU times: user 262 ms, sys: 22.1 ms, total: 285 ms
Wall time: 1.82 s


In [22]:
df[df["state_name"] == "Pennsylvania"].tail()

,candidate,party,date,pct_estimate,election,state_name
2565,Fetterman,DEM,2022-06-23,48.2,2022-11-08,Pennsylvania
2566,Fetterman,DEM,2022-06-22,48.2,2022-11-08,Pennsylvania
2567,Fetterman,DEM,2022-06-21,48.2,2022-11-08,Pennsylvania
2568,Fetterman,DEM,2022-06-20,48.2,2022-11-08,Pennsylvania
2569,Fetterman,DEM,2022-06-19,48.2,2022-11-08,Pennsylvania


---

#### Chart it

In [24]:
alt.Chart(
    df[(df["date"] >= "2022-07-01") & (df["party"].str.contains("REP|DEM"))]
).mark_line().encode(
    x=alt.X(
        "date:T",
        title="",
        axis=alt.Axis(format="%b. %-d", tickCount=2),
    ),
    y=alt.Y("pct_estimate", title="", scale=alt.Scale(domain=[40, 60])),
    color=alt.Color(
        "party",
        legend=None,
        scale=alt.Scale(domain=["REP", "DEM"], range=["red", "blue"]),
    ),
    facet=alt.Facet("state_name", title="", columns=4),
).properties(
    width=300, height=200
).configure_legend(
    orient="top", symbolType="stroke"
)

alt.Chart(...)

In [23]:
chart = (
    alt.Chart()
    .mark_line(color="#1851ac")
    .encode(
        x=alt.X("display_date:O", title="", sort="-x"),
        y=alt.Y("dem_polling_margin", title="", axis=alt.Axis()),
    )
)

text = (
    alt.Chart()
    .mark_text(dx=0, dy=-8, color="black")
    .encode(
        x=alt.X("display_date", sort="-x"),
        y=alt.Y("dem_polling_margin"),
        text=alt.Text("dem_polling_margin"),
    )
)

alt.layer(chart, text, data=df,).properties(width=100, height=120,).facet(
    facet=alt.Facet("state_name", title=" "), columns=7
).properties(
    title="Percentage point change in Democrats' polling average margin since Oct. 5"
)

NameError: name 'thennow_df' is not defined